In [2]:
import pandas as pd
import rdkit
from chembl_webresource_client.new_client import new_client

In [26]:
# Создаем API

activity_api= new_client.activity.filter(target_chembl_id='CHEMBL230', type='IC50').only('molecule_chembl_id','canonical_smiles', 'standard_value', 'units')

# Еще немного фильтруем ()

molecule_data = activity_api.filter(type='IC50', standard_value__lte = 1000)

molecule_data = list(molecule_data)

# Приведение единиц измерения к одному виду

l = len(molecule_data)
for data in molecule_data:
    unit = data['units']
    if unit == 'uM':
        data['standard_value'] = float(data['standard_value'])*1000
        data['units'] = 'nM'
    elif unit == 'umol/L':
        data['standard_value'] = float(data['standard_value'])*1000
        data['units'] = 'nM'
    elif unit == "10'-8M":
        data['standard_value'] = float(data['standard_value'])/10
        data['units'] = 'nM'
    else:
        continue

# Проверка корректности SMILES молекулы

for data in molecule_data:
    smiles = data['canonical_smiles']
    if smiles is None:
        molecule_data.remove(data)    
    elif rdkit.Chem.MolFromSmiles(smiles) is None:
        molecule_data.remove(data)
    
# Создание DataFrame

df = pd.DataFrame(molecule_data)

# Фильтрация от дубликатов, фильтрация по 'standard_value' и по 'units'

df = pd.DataFrame.drop_duplicates(df, subset='canonical_smiles')
df['standard_value'] = pd.to_numeric(df['standard_value'], errors='coerce')
df = df[df['standard_value'] <= 1000]
df = df[df['units'] == 'nM']

# Запись результата в .csv файл

with open('Parsing_res.csv', 'w', newline='', encoding='utf-8'):
    df.to_csv('Parsing_res.csv', columns = ['molecule_chembl_id','canonical_smiles', 'standard_value', 'units'], index = False)

